In [35]:
import scipy.io as sio
import numpy as np
from matplotlib.colors import ListedColormap
from scipy.stats import pearsonr
import seaborn as sn
import mne
import matplotlib.pyplot as plt
%matplotlib qt
%gui qt

# File names for the baseline and signal files
str_FileName_R = 'Sujeto14_R_processed.fif'  # Name of the baseline file
str_FileName_MF = 'Sujeto14_MF_processed.fif'  # Name of the signal file
fileR = mne.read_epochs(str_FileName_R)  
fileMF = mne.read_epochs(str_FileName_MF)

# Get the data 
data_R = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
data_MF = fileMF.get_data()
normalized_data = np.zeros_like(data_MF)

Reading c:\Users\afcad\Downloads\Sujeto14_R_processed.fif ...
    Found the data of interest:


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\1828133967.py:14: RuntimeWarning: This filename (Sujeto14_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(str_FileName_R)


        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
71 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto14_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\1828133967.py:15: RuntimeWarning: This filename (Sujeto14_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
141 matching events found
No baseline correction applied
0 projection items activated


In [38]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Average PSD across all epochs
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
}

# Plot topomaps for resting state
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the power across the band
    band_power_R = np.mean(avg_psd_R[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_R, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Resting)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')
    
plt.suptitle("Resting State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

# Plot topomaps for mindfulness state
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_MF >= fmin, freqs_MF <= fmax)
    
    # Average the power across the band
    band_power_MF = np.mean(avg_psd_MF[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap
    im, _ = mne.viz.plot_topomap(
        band_power_MF, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False  # Disable immediate showing to allow subplot arrangement
    )
    ax.set_title(f'{band} Band (Mindfulness)')
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power')

plt.suptitle("Mindfulness State Topomap", fontsize=16)
plt.tight_layout()
plt.show()

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


In [39]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Average PSD across all windows (mean over epochs axis, axis=0)
avg_psd_R = np.mean(psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_psd_MF = np.mean(psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Calculate the difference
difference_psd = avg_psd_MF - avg_psd_R  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'Fast Beta': (18, 30),
}

# Calculate global min and max across all bands
global_min = np.inf
global_max = -np.inf

for fmin, fmax in frequency_bands.values():
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over frequencies
    global_min = min(global_min, band_power_difference.min())
    global_max = max(global_max, band_power_difference.max())

# Extract band-specific differences and prepare the figure
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
    # Find indices for the current band
    band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
    # Average the difference across the band
    band_power_difference = np.mean(difference_psd[:, band_idx], axis=1)  # Mean over the frequency axis
    
    # Plot the topomap with consistent vlim
    im, _ = mne.viz.plot_topomap(
        band_power_difference, 
        info, 
        axes=ax, 
        cmap='jet', 
        show=False,  # Disable immediate showing to allow subplot arrangement
        names=fileR.ch_names,
    #   vlim=(global_min, global_max)  # Use global min and max
    )
    ax.set_title(f"{band} Band")
    # Add a colorbar for the subplot
    cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
    cbar.set_label('Power Difference')

# Finalize and display the figure
plt.suptitle("Mindfulness Difference", fontsize=16)
plt.tight_layout()
plt.show()


Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


In [5]:
# import numpy as np
# import mne
# import matplotlib.pyplot as plt

# # Compute PSDs using Welch's method
# psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
# psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# # Normalize PSDs (relative power for each channel)
# total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
# total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
# relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
# relative_psd_MF = psd_MF / total_power_MF

# # Average relative PSDs across epochs
# avg_relative_psd_R = np.mean(relative_psd_R, axis=0)  # Shape: (n_channels, n_freqs)
# avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# # Compute the relative power difference
# relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# # Define frequency bands
# frequency_bands = {
#     'Delta': (0.5, 4),
#     'Theta': (4, 8),
#     'Alpha': (8, 12),
#     'Beta': (12, 18),
#     'FastBeta': (18, 30),
# }

# # Prepare the figure for topomaps
# info = fileR.info  # Use the baseline's info for channel locations
# n_bands = len(frequency_bands)
# fig, axes = plt.subplots(1, n_bands, figsize=(15, 5))  # Create a row of subplots

# for ax, (band, (fmin, fmax)) in zip(axes, frequency_bands.items()):
#     # Find indices for the current band
#     band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
    
#     # Average the relative power difference across the band
#     band_relative_power_diff = np.mean(relative_power_difference[:, band_idx], axis=1)  # Mean over frequencies
   
#     # Plot the topomap with a colorbar
#     im, _ = mne.viz.plot_topomap(
#         band_relative_power_diff, 
#         info, 
#         axes=ax, 
#         cmap='jet', 
#         show=False  # Disable immediate showing to allow subplot arrangement
#     )
#     ax.set_title(f'{band} Band')
    
#     # Add a colorbar for the subplot
#     cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
#     cbar.set_label('Relative Power Difference')

# # Finalize and display the figure
# plt.tight_layout()
# plt.show()

## Resting, Mind, DiffNormalized

In [24]:
import numpy as np
import mne
import matplotlib.pyplot as plt

# Compute PSDs using Welch's method
psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)
psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=0.5, fmax=30).get_data(return_freqs=True)

# Normalize PSDs (relative power for each channel)
total_power_R = np.sum(psd_R, axis=2, keepdims=True)  # Total power across frequencies per channel for R
total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)  # Total power across frequencies per channel for MF
relative_psd_R = psd_R / total_power_R  # Shape: (n_epochs, n_channels, n_freqs)
relative_psd_MF = psd_MF / total_power_MF

# Average relative PSDs across epochs
avg_relative_psd_R = np.mean(relative_psd_R, axis=0)  # Shape: (n_channels, n_freqs)
avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)  # Shape: (n_channels, n_freqs)

# Compute the relative power difference
relative_power_difference = avg_relative_psd_MF - avg_relative_psd_R  # Shape: (n_channels, n_freqs)

# Define frequency bands
frequency_bands = {
    'Delta': (0.5, 4),
    'Theta': (4, 8),
    'Alpha': (8, 12),
    'Beta': (12, 18),
    'FastBeta': (18, 30),
}

# Prepare the figure for topomaps
info = fileR.info  # Use the baseline's info for channel locations
n_bands = len(frequency_bands)
fig, axes = plt.subplots(3, n_bands, figsize=(11, 12),gridspec_kw={'wspace': 0.0001, 'hspace': 0.0001})  # Adjust spacing  # Create a grid of subplots

for row, data, title in zip(
    range(3),
    [avg_relative_psd_R, avg_relative_psd_MF, relative_power_difference],
    ['Resting', 'Mindfulness', 'Normalized Difference']
):
    for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
        # Find indices for the current band
        band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
        
        # Average the data across the band
        band_data = np.mean(data[:, band_idx], axis=1)  # Mean over frequencies
        
        # Plot the topomap with a colorbar
        im, _ = mne.viz.plot_topomap(
            band_data, 
            info, 
            axes=ax, 
            cmap='jet', 
            show=False,  # Disable immediate showing to allow subplot arrangement
        )
        ax.set_title(f'{band} Band' if row == 0 else '')
        
        # Add a colorbar for the subplot
        cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
        if row == 2:  # Label the colorbar only for the last row
            cbar.set_label(title)

# Add row titles
for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
    ax.annotate(
        title, 
        xy=(0, 0.5), 
        xytext=(-ax.yaxis.labelpad - 5, 0),
        xycoords=ax.yaxis.label, 
        textcoords='offset points',
        size='large', ha='right', va='center', rotation=90
    )

# Finalize and display the figure
plt.tight_layout()
plt.show()

Effective window size : 2.000 (s)
Effective window size : 2.000 (s)


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\413725337.py:75: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## For Several files (relative)

In [88]:
def topoplotsAverage(file_pairs, fmin=0.5, fmax=30):
    """
    Compute and plot the average PSDs and their differences for multiple pairs of EEG files,
    ensuring all subplots share the same color scale.

    Parameters:
    - file_pairs: List of tuples [(fileR1, fileMF1), (fileR2, fileMF2), ...]
                  Each tuple contains a resting state file and a mindfulness state file.
    - fmin: Minimum frequency for PSD calculation (default: 0.5 Hz).
    - fmax: Maximum frequency for PSD calculation (default: 30 Hz).

    Returns:
    - None (plots the results).
    """
    avg_psd_R_all = []
    avg_psd_MF_all = []

    for str_FileName_R, str_FileName_MF in file_pairs:
        fileR = mne.read_epochs(str_FileName_R)  
        fileMF = mne.read_epochs(str_FileName_MF)
        
        # Compute PSDs using Welch's method
        psd_R, freqs_R = fileR.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)
        psd_MF, freqs_MF = fileMF.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

        # Normalize PSDs (relative power for each channel)
        total_power_R = np.sum(psd_R, axis=2, keepdims=True)
        total_power_MF = np.sum(psd_MF, axis=2, keepdims=True)
        relative_psd_R = psd_R / total_power_R
        relative_psd_MF = psd_MF / total_power_MF

        # Average relative PSDs across epochs
        avg_relative_psd_R = np.mean(relative_psd_R, axis=0)
        avg_relative_psd_MF = np.mean(relative_psd_MF, axis=0)

        avg_psd_R_all.append(avg_relative_psd_R)
        avg_psd_MF_all.append(avg_relative_psd_MF)

    # Compute grand averages across all pairs
    avg_psd_R_all = np.mean(avg_psd_R_all, axis=0)
    avg_psd_MF_all = np.mean(avg_psd_MF_all, axis=0)

    # Compute the relative power difference
    relative_power_difference = avg_psd_MF_all - avg_psd_R_all

    # Define frequency bands
    frequency_bands = {
        'Delta': (0.5, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 18),
        'FastBeta': (18, 30),
    }

    # Calculate global min and max for the color scale across all bands
    global_min = np.inf
    global_max = -np.inf

    for fmin, fmax in frequency_bands.values():
        band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)
        band_data = np.mean(relative_power_difference[:, band_idx], axis=1)  # Average over band frequencies
        global_min = min(global_min, band_data.min())
        global_max = max(global_max, band_data.max())

    # Prepare the figure for topomaps
    info = mne.read_epochs(file_pairs[0][0]).info  # Use the info from the first resting state file
    n_bands = len(frequency_bands)
    fig, axes = plt.subplots(3, n_bands, figsize=(11, 12), gridspec_kw={'wspace': 0.4, 'hspace': 0.4})

    for row, data, title in zip(
        range(3),
        [avg_psd_R_all, avg_psd_MF_all, relative_power_difference],
        ['Resting', 'Mindfulness', 'Normalized Difference']
    ):
        for ax, (band, (fmin, fmax)) in zip(axes[row], frequency_bands.items()):
            # Find indices for the current band
            band_idx = np.logical_and(freqs_R >= fmin, freqs_R <= fmax)

            # Average the data across the band
            band_data = np.mean(data[:, band_idx], axis=1)

            # Plot the topomap with shared vmin and vmax
            im, _ = mne.viz.plot_topomap(
                band_data,
                info,
                axes=ax,
                cmap='jet',
                show=False,
                #vlim=(global_min, global_max) 
            )
            ax.set_title(f'{band} Band' if row == 0 else '')

            # Add a colorbar for the subplot
            cbar = plt.colorbar(im, ax=ax, shrink=0.6, orientation='horizontal', pad=0.1)
            if row == 2:
                cbar.set_label(title)

    # Add row titles
    for ax, title in zip(axes[:, 0], ['Resting', 'Mindfulness', 'Normalized Difference']):
        ax.annotate(
            title,
            xy=(0, 0.5),
            xytext=(-ax.yaxis.labelpad - 5, 0),
            xycoords=ax.yaxis.label,
            textcoords='offset points',
            size='large', ha='right', va='center', rotation=90
        )

    # Finalize and display the figure
    plt.tight_layout()
    plt.show()

file_pairs = [
    ('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif'),
    ('Sujeto9_R_processed.fif', 'Sujeto9_MF_processed.fif'),
    ('Sujeto11_R_processed.fif', 'Sujeto11_MF_processed.fif'),
    ('Sujeto14_R_processed.fif', 'Sujeto14_MF_processed.fif'),
    ('Sujeto16_R_processed.fif', 'Sujeto16_MF_processed.fif'),
]

topoplotsAverage(file_pairs)


Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\2898196739.py:19: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(str_FileName_R)


Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto6_MF_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\2898196739.py:20: RuntimeWarning: This filename (Sujeto6_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


...

C:\Users\afcad\AppData\Local\Temp\ipykernel_580\2898196739.py:20: RuntimeWarning: This filename (Sujeto16_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(str_FileName_MF)


Not setting metadata
106 matching events found
No baseline correction applied
0 projection items activated
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...
    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\2898196739.py:66: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  info = mne.read_epochs(file_pairs[0][0]).info  # Use the info from the first resting state file


Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\2898196739.py:110: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


## Flattenizado

In [52]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Extract data
dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)

# Sampling frequency
sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

# Trim the last 0.5 seconds
dataR_trimmed = dataR[:, :, :n_times_to_keep]
dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

# Compute mean and standard deviation of the trimmed resting signal
mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

# Concatenate both signals along epochs
data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

# Normalize the concatenated signal
data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None]

# Concatenate all epochs along the time axis for each channel
data_normalized_ch = np.hstack(data_normalized)

print(np.shape(data_normalized_ch))

# Assuming data_normalized_ch has shape (n_channels, total_time)
n_channels, total_time = data_normalized_ch.shape
time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
electrode_names = fileR.ch_names  # List of electrode names

concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed
# Create the heatmap
plt.figure(figsize=(12, 6))
plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
           extent=[time[0], time[-1], n_channels, 0])  # Extent flips the y-axis to show first channel at the top

# Add a vertical line at the concatenation point
plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

# Add labels and colorbar
plt.colorbar(label='Z-score')
plt.xlabel('Time (s)')
plt.ylabel('Electrodes')
plt.title('Heatmap of Normalized Signal Amplitudes')
plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
plt.show()

(61, 728222)


In [53]:
# Slice the data to include only the first 5 electrodes
data_subset = data_normalized_ch[:5, :]  # Shape: (5, total_time)

# Create a figure with subplots for heatmap and signals
fig, axes = plt.subplots(6, 1, figsize=(12, 12), gridspec_kw={'height_ratios': [5, 1, 1, 1, 1, 1]}, sharex=True)

# Plot the heatmap in the first subplot
im = axes[0].imshow(data_subset, aspect='auto', cmap='viridis',
                    extent=[time[0], time[-1], 5, 0])
# Add a vertical line at the concatenation point
axes[0].axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

axes[0].set_ylabel('Electrodes')
axes[0].set_title('Heatmap of Normalized Signal Amplitudes (First 5 Electrodes)')
axes[0].yaxis.set_ticks(range(5))
axes[0].set_yticklabels([f'Electrode {i+1}' for i in range(5)])
plt.colorbar(im, ax=axes[0], orientation='horizontal', fraction=0.05, pad=0.1, label='Amplitude (normalized)')

# Plot signals in the next 5 subplots (one for each electrode)
for i in range(5):
    axes[i + 1].plot(time, data_subset[i], label=f'Electrode {i+1}')
    axes[i + 1].set_ylabel('Amplitude')
    axes[i + 1].legend(loc='upper right')

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()


## For several Files signal vs time

In [80]:
import mne
import numpy as np
import matplotlib.pyplot as plt

def process_and_visualize_eeg(file_R, file_MF):
    """
    Process two EEG files (resting and MF signals), normalize them, and create a heatmap.
    
    Parameters:
        file_R (str): Filepath to the resting signal (.fif).
        file_MF (str): Filepath to the MF signal (.fif).
        
    Returns:
        np.ndarray: Normalized concatenated matrix of shape (n_channels, total_time).
    """
    # Load files
    fileR = mne.read_epochs(file_R)  
    fileMF = mne.read_epochs(file_MF)

    # Extract data
    dataR = fileR.get_data()  # Shape: (n_epochs, n_channels, n_times)
    dataMF = fileMF.get_data()  # Shape: (n_epochs, n_channels, n_times)

    # Sampling frequency
    sfreq = fileR.info['sfreq']  # Sampling frequency (Hz)
    n_times_to_keep = int(dataR.shape[2] - 0.5 * sfreq)  # Time points to keep

    # Trim the last 0.5 seconds
    dataR_trimmed = dataR[:, :, :n_times_to_keep]
    dataMF_trimmed = dataMF[:, :, :n_times_to_keep]

    # Compute mean and standard deviation of the trimmed resting signal
    mean_R = np.mean(dataR_trimmed, axis=(0, 2))  # Mean across epochs and time
    std_R = np.std(dataR_trimmed, axis=(0, 2))    # Std across epochs and time

    # Concatenate both signals along epochs
    data_combined = np.concatenate([dataR_trimmed, dataMF_trimmed], axis=0)

    # Normalize the concatenated signal
    data_normalized = (data_combined - mean_R[None, :, None]) / std_R[None, :, None]

    # Concatenate all epochs along the time axis for each channel
    data_normalized_ch = np.hstack(data_normalized)

    # Visualization
    n_channels, total_time = data_normalized_ch.shape
    time = np.linspace(0, total_time / sfreq, total_time)  # Generate time axis based on sampling frequency
    electrode_names = fileR.ch_names  # List of electrode names

    # Compute concatenation point in seconds
    concat_point = dataR_trimmed.shape[0] * dataR_trimmed.shape[2] / sfreq  # Total time of dataR_trimmed

    # Create the heatmap
    plt.figure(figsize=(12, 6))
    plt.imshow(data_normalized_ch, aspect='auto', cmap='viridis', 
               extent=[time[0], time[-1], n_channels, 0])  # Extent flips the y-axis to show the first channel at the top

    # Add a vertical line at the concatenation point
    plt.axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

    # Add labels and colorbar
    plt.colorbar(label='Z-score')
    plt.xlabel('Time (s)')
    plt.ylabel('Electrodes')
    plt.title('Heatmap of Normalized Signal Amplitudes')
    plt.yticks(ticks=np.arange(len(electrode_names)), labels=electrode_names)
    plt.legend()
    plt.show()

    return data_normalized_ch

normalized_matrix = process_and_visualize_eeg('Sujeto6_R_processed.fif', 'Sujeto6_MF_processed.fif')
print("Normalized matrix shape:", normalized_matrix.shape)


Processing Sujeto6_R_processed.fif and Sujeto6_MF_processed.fif...
Reading c:\Users\afcad\Downloads\Sujeto6_R_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\29792418.py:17: RuntimeWarning: This filename (Sujeto6_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(file_R)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
72 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto6_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\29792418.py:18: RuntimeWarning: This filename (Sujeto6_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(file_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
107 matching events found
No baseline correction applied
0 projection items activated
Processing Sujeto9_R_processed.fif and Sujeto9_MF_processed.fif...
Reading c:\Users\afcad\Downloads\Sujeto9_R_processed.fif ...
    Found the data of interest:


...

C:\Users\afcad\AppData\Local\Temp\ipykernel_580\29792418.py:17: RuntimeWarning: This filename (Sujeto16_R_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileR = mne.read_epochs(file_R)


Not setting metadata
52 matching events found
No baseline correction applied
0 projection items activated
Reading c:\Users\afcad\Downloads\Sujeto16_MF_processed.fif ...


C:\Users\afcad\AppData\Local\Temp\ipykernel_580\29792418.py:18: RuntimeWarning: This filename (Sujeto16_MF_processed.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  fileMF = mne.read_epochs(file_MF)


    Found the data of interest:
        t =   -2500.00 ...    2500.00 ms
        0 CTF compensation matrices available
Not setting metadata
106 matching events found
No baseline correction applied
0 projection items activated


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 61) + inhomogeneous part.

In [60]:
# Select the first 5 averaged channels
data_subset = avg_data_by_channel[:5, :]  # Shape: (5, total_time)

# Create a figure with subplots for heatmap and individual signals
fig, axes = plt.subplots(6, 1, figsize=(12, 12), gridspec_kw={'height_ratios': [5, 1, 1, 1, 1, 1]}, sharex=True)

# Plot the heatmap in the first subplot
im = axes[0].imshow(data_subset, aspect='auto', cmap='viridis',
                    extent=[time[0], time[-1], 5, 0])
# Add a vertical line at the concatenation point
axes[0].axvline(x=concat_point, color='black', linestyle='--', label='Concatenation Point')

axes[0].set_ylabel('Electrodes')
axes[0].set_title('Heatmap of Normalized Signal Amplitudes (First 5 Averaged Electrodes)')
axes[0].yaxis.set_ticks(range(5))
axes[0].set_yticklabels([all_channel_names[i] for i in range(5)])  # Use actual electrode names
plt.colorbar(im, ax=axes[0], orientation='horizontal', fraction=0.05, pad=0.1, label='Amplitude (normalized)')

# Plot signals in the next 5 subplots (one for each electrode)
for i in range(5):
    axes[i + 1].plot(time, data_subset[i], label=f'{all_channel_names[i]}')
    axes[i + 1].set_ylabel('Amplitude')
    axes[i + 1].legend(loc='upper right')

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()